# Blockchain API: Get data for a single address

In [1]:
import pandas as pd
import altair as alt

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)

### Call url

In [3]:
url = 'https://blockchain.info/rawaddr/1Bm9Eqyd46qhJH8pQi7W8QxDA5VhbtKbXK?format=json&limit=1000'

### Read the results

In [4]:
src = pd.read_json(url)

### How many records? 

In [5]:
len(src)

415

### What's the bitcoin exchange rate?

In [6]:
rate = pd.read_json('https://blockchain.info/ticker')
blockchain_rate = rate['USD']['last']

### The transaction specific data are in a nexted column called 'txs'

In [7]:
txs = pd.json_normalize(src['txs'])

### Deal with dates, times

In [8]:
txs['datetime'] = pd.to_datetime(txs['time'], unit='s')

In [9]:
txs['date'] = txs['datetime'].dt.date
txs['time'] = txs['datetime'].dt.time

### Only include fields that seem descriptive

In [10]:
txs_slim = txs[["datetime", 'date', 'time', 'result', 'balance', 'fee', 'hash', 'ver', 'vin_sz', 'vout_sz', 'size', 'weight', 'tx_index',
       'block_index', 'block_height', ]].copy()

In [11]:
txs_slim[['result', 'balance']] = txs_slim[['result', 'balance']].astype(float)

In [12]:
txs_slim['new_result_btc'] = txs_slim['result'] / 100000000
txs_slim['new_balance_btc'] = txs_slim['balance'] / 100000000

In [13]:
txs_slim['new_result_usd'] = (txs_slim['new_result_btc'] * blockchain_rate).round(2)
txs_slim['new_balance_usd'] = (txs_slim['new_balance_btc'] * blockchain_rate).round(2)

---

### Aggregate

In [14]:
txs_slim_grp = txs_slim.groupby(['date']).agg({'hash': 'count', 'fee':['mean', 'sum']}).round(2).reset_index()

In [15]:
txs_slim_grp.columns = ['_'.join(col).strip() for col in txs_slim_grp.columns.values]

In [16]:
txs_slim_grp.rename(columns={'date_':'date', 'hash_count': 'transaction_count'}, inplace=True)

In [17]:
txs_slim_grp['date'] = pd.to_datetime(txs_slim_grp['date'])

---

### Average daily transaction count

In [18]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='transaction_count'
)

alt.Chart(...)

### Average daily transaction fee (not sure what this fee means yet)

In [19]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='fee_mean'
)

alt.Chart(...)

---

### Export

In [20]:
txs_slim.to_csv('data/raw/transactions.csv', index=False)